In [2]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from flask_.QueryHandler import QueryHandler

In [2]:
opening_lda = OpeningLDA(num_passes=15) 

print(len(opening_lda.texts.keys()))
opening_lda.lda_model.print_topics(num_words=15)

opening_lda.save('lda2')


369


In [3]:
query_handler = QueryHandler(load_from_file='lda1')

9694


In [8]:
sample_query1 = "capture the center, develop light square bishop"

In [9]:
query_handler.handle_user_query(sample_query1)

Rel index matches:

D43 : 0.17293233082706766
B00 : 0.1415929203539823
B01 : 0.13821138211382114
A51 : 0.03968253968253968
D85 : 0.0390625
A09 : 0.03787878787878788
B80 : 0.037037037037037035
D07 : 0.03333333333333333
B18 : 0.030303030303030304
C42 : 0.03007518796992481
E41 : 0.03007518796992481
B06 : 0.029850746268656716
D90 : 0.029850746268656716
B36 : 0.02962962962962963
D20 : 0.02877697841726619
C43 : 0.028368794326241134
A00 : 0.028037383177570093
C10 : 0.026785714285714284
D30 : 0.026785714285714284
A20 : 0.02586206896551724
C02 : 0.02459016393442623
C31 : 0.022900763358778626
C47 : 0.022900763358778626
B60 : 0.022556390977443608
E32 : 0.022388059701492536
D82 : 0.021739130434782608
B41 : 0.021739130434782608
A60 : 0.021739130434782608
C03 : 0.02142857142857143
C36 : 0.02097902097902098
E42 : 0.02054794520547945
C50 : 0.020202020202020204
B02 : 0.020172910662824207
B12 : 0.02
B90 : 0.019736842105263157
B70 : 0.01935483870967742
E76 : 0.019230769230769232
C45 : 0.0189873417721519
